<a href="https://colab.research.google.com/github/kalepravinkumars/RSI_MA_NSE/blob/ITR_1/ITR_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance as yf

def calculate_rsi(data, period=14):
    delta = data.diff().dropna()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    avg_gain = gains.rolling(window=period).mean()
    avg_loss = losses.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_ema(data, period):
    return data.ewm(span=period).mean()

# Fetch stock data from Yahoo Finance
symbol = "SBIN.NS"  # Symbol for SBI stock on NSE
stock_data = yf.download(symbol, start="2019-01-01", end="2022-01-01")

# Calculate RSI
stock_data['RSI'] = calculate_rsi(stock_data['Close'], period=13)

# Calculate EMAs for stock data
ema_periods = [13, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]
for period in ema_periods:
    column_name = f'EMA_{period}'
    stock_data[column_name] = calculate_ema(stock_data['Close'], period)

# Calculate EMAs for RSI data
rsi_ema_periods = [55, 35]
for period in rsi_ema_periods:
    column_name = f'RSI_EMA_{period}'
    stock_data[column_name] = calculate_ema(stock_data['RSI'], period)

# Fill missing values with 0
stock_data.fillna(0, inplace=True)

# Print the updated stock data
stock_data.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,RSI,EMA_13,EMA_8,EMA_21,EMA_34,EMA_55,EMA_89,EMA_144,EMA_233,EMA_377,EMA_610,EMA_987,RSI_EMA_55,RSI_EMA_35
Date,,,,,,,,,,,,,,,,,,,,
2019-01-01,297.500000,300.700012,293.850006,299.600006,286.725800,11837127,0.0,299.600006,299.600006,299.600006,299.600006,299.600006,299.600006,299.600006,299.600006,299.600006,299.600006,299.600006,0.0,0.0
2019-01-02,299.100006,302.500000,293.100006,293.899994,281.270721,25559853,0.0,296.530769,296.393749,296.614285,296.666176,296.698182,296.717977,296.730208,296.737768,296.742440,296.745328,296.747112,0.0,0.0
2019-01-03,295.000000,295.549988,290.100006,291.100006,278.591064,17548347,0.0,294.435435,294.172023,294.598491,294.700607,294.763863,294.803082,294.827349,294.842360,294.851642,294.857381,294.860928,0.0,0.0
2019-01-04,292.100006,299.000000,291.500000,297.649994,284.859558,19514041,0.0,295.433257,295.390985,295.473637,295.504251,295.525220,295.538979,295.547779,295.553332,295.556807,295.558971,295.560315,0.0,0.0
2019-01-07,301.049988,301.500000,295.200012,296.299988,283.567596,14579399,0.0,295.663688,295.673356,295.671809,295.682655,295.691644,295.698097,295.702428,295.705238,295.707024,295.708148,295.708850,0.0,0.0


In [5]:
import pandas as pd
import numpy as np

# Generate random buy and sell signals
np.random.seed(42)  # Set random seed for reproducibility
trading_signals = np.random.randint(0, 2, size=len(stock_data))  # Generate random 0s and 1s for signals

# Initialize trading log dataframe
trading_log = pd.DataFrame(stock_data[['Open', 'Close', 'High', 'Low']], columns=['Open', 'Close', 'High', 'Low'])
trading_log['Buy Signal'] = 0
trading_log['Sell Signal'] = 0
trading_log['Position'] = False
trading_log['Profit/Loss'] = 0.0

# Apply random buy and sell signals and calculate profit/loss
position = False
buy_price = 0.0
for i in range(len(stock_data)):
    if trading_signals[i] == 1:  # Buy signal
        if not position:  # Enter new position
            position = True
            buy_price = stock_data['Close'].iloc[i]
            trading_log.at[i, 'Buy Signal'] = 1
            trading_log.at[i, 'Position'] = True
    elif trading_signals[i] == 0:  # Sell signal
        if position:  # Exit current position
            position = False
            sell_price = stock_data['Close'].iloc[i]
            trading_log.at[i, 'Sell Signal'] = 1
            trading_log.at[i, 'Position'] = False
            trading_log.at[i, 'Profit/Loss'] = sell_price - buy_price

# Print the trading log
trading_log.head()


,Open,Close,High,Low,Buy Signal,Sell Signal,Position,Profit/Loss
Date,,,,,,,,
2019-01-01 00:00:00,297.500000,299.600006,300.700012,293.850006,0.0,0.0,False,0.0
2019-01-02 00:00:00,299.100006,293.899994,302.500000,293.100006,0.0,0.0,False,0.0
2019-01-03 00:00:00,295.000000,291.100006,295.549988,290.100006,0.0,0.0,False,0.0
2019-01-04 00:00:00,292.100006,297.649994,299.000000,291.500000,0.0,0.0,False,0.0
2019-01-07 00:00:00,301.049988,296.299988,301.500000,295.200012,0.0,0.0,False,0.0


In [6]:
# Print the trading log
trading_log.tail()

,Open,Close,High,Low,Buy Signal,Sell Signal,Position,Profit/Loss
Date,,,,,,,,
736,NaN,NaN,NaN,NaN,1.0,NaN,True,NaN
737,NaN,NaN,NaN,NaN,NaN,1.0,False,1.099976
739,NaN,NaN,NaN,NaN,1.0,NaN,True,NaN
740,NaN,NaN,NaN,NaN,NaN,1.0,False,-2.699982
741,NaN,NaN,NaN,NaN,1.0,NaN,True,NaN


In [10]:
trading_log[trading_log['Profit/Loss']>0]

,Open,Close,High,Low,Buy Signal,Sell Signal,Position,Profit/Loss
Date,,,,,,,,
10,NaN,NaN,NaN,NaN,NaN,1.0,False,1.250000
21,NaN,NaN,NaN,NaN,NaN,1.0,False,6.850006
35,NaN,NaN,NaN,NaN,NaN,1.0,False,4.750000
37,NaN,NaN,NaN,NaN,NaN,1.0,False,2.950012
47,NaN,NaN,NaN,NaN,NaN,1.0,False,14.399994
...,...,...,...,...,...,...,...,...
704,NaN,NaN,NaN,NaN,NaN,1.0,False,5.900024
721,NaN,NaN,NaN,NaN,NaN,1.0,False,8.049988
723,NaN,NaN,NaN,NaN,NaN,1.0,False,11.400024


In [12]:
import pandas as pd
import numpy as np

# Function to generate random buy and sell signals
def generate_random_signals(data):
    random_signals = np.random.choice([0, 1], size=len(data))
    return random_signals

# Create a trading log dataframe
trading_log = pd.DataFrame()

# Copy relevant columns from stock_data to trading_log
trading_log['Date'] = stock_data.index
trading_log['RSI'] = stock_data['RSI']
trading_log['Close'] = stock_data['Close']
trading_log['RSI_EMA_55'] = stock_data['RSI_EMA_55']
trading_log['RSI_EMA_35'] = stock_data['RSI_EMA_35']
trading_log['EMA_13'] = stock_data['EMA_13']
trading_log['EMA_8'] = stock_data['EMA_8']
trading_log['EMA_21'] = stock_data['EMA_21']
trading_log['EMA_34'] = stock_data['EMA_34']
trading_log['EMA_55'] = stock_data['EMA_55']
trading_log['EMA_89'] = stock_data['EMA_89']
trading_log['EMA_144'] = stock_data['EMA_144']
trading_log['EMA_233'] = stock_data['EMA_233']
trading_log['EMA_377'] = stock_data['EMA_377']
trading_log['EMA_610'] = stock_data['EMA_610']
trading_log['EMA_987'] = stock_data['EMA_987']

# Generate random buy and sell signals
trading_log['Signal'] = generate_random_signals(trading_log)

# Initialize position and profit/loss
position = False
profit_loss = 0

# Iterate through the trading log and calculate position and profit/loss
for index, row in trading_log.iterrows():
    if row['Signal'] == 1:  # Buy signal
        if not position:  # Enter a new position if not already in one
            position = True
            buy_price = row['Close']
    elif row['Signal'] == 0:  # Sell signal
        if position:  # Exit the position if already in one
            position = False
            sell_price = row['Close']
            trade_profit_loss = sell_price - buy_price
            profit_loss += trade_profit_loss
            trading_log.at[index, 'Profit/Loss'] = trade_profit_loss
    trading_log.at[index, 'Position'] = position

# Print the trading log
print(trading_log)


KeyError: ignored